In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

from prediction_flow.features import Number, Category, Sequence, Features
from prediction_flow.transformers.column import (
    StandardScaler, CategoryEncoder, SequenceEncoder)

from prediction_flow.pytorch.data import Dataset
from prediction_flow.pytorch import DeepFM, DNN, DIN, DIEN, AttentionGroup

from prediction_flow.pytorch.functions import fit, predict, create_dataloader_fn

In [2]:
train_df = pd.read_csv("./ml-1m/train.csv")
valid_df = pd.read_csv("./ml-1m/test.csv")

In [3]:
train_df.histHighRatedMovieIds = train_df.histHighRatedMovieIds.astype(str)
valid_df.histHighRatedMovieIds = valid_df.histHighRatedMovieIds.astype(str)

In [4]:
train_df.dtypes

userId                     int64
movieId                    int64
ratting                    int64
timestamp                  int64
last                     float64
histHighRatedMovieIds     object
gender                    object
age                        int64
occupation                 int64
zipCode                   object
title                     object
genres                    object
label                      int64
dtype: object

In [5]:
valid_df.dtypes

userId                     int64
movieId                    int64
ratting                    int64
timestamp                  int64
last                     float64
histHighRatedMovieIds     object
gender                    object
age                        int64
occupation                 int64
zipCode                   object
title                     object
genres                    object
label                      int64
dtype: object

In [6]:
train_df.head()

,userId,movieId,ratting,timestamp,last,histHighRatedMovieIds,gender,age,occupation,zipCode,title,genres,label
0,1,48,5,978824351,1.0,1545|527|595|588|1|2355|2294|783|1566|1907,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance,1
1,2030,48,3,977809337,1.0,3928|1934|2099|3345|1013|1101|3524|3429|3751|2138,M,25,4,77345,Pocahontas (1995),Animation|Children's|Musical|Romance,0
2,4877,48,2,962767153,1.0,2078|1032|2081|616|364|2033|3034|2394|3745|3615,M,25,4,94703,Pocahontas (1995),Animation|Children's|Musical|Romance,0
3,2,1917,3,978300174,1.0,3418|349|1527|2353|1370|648|368|736|2002|1544,M,56,16,70072,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,0
4,5122,1917,5,962213528,1.0,3078|2858|2907|1584|2571|32|788|1573|2428|1580,M,25,0,20009,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,1


In [7]:
valid_df.head()

,userId,movieId,ratting,timestamp,last,histHighRatedMovieIds,gender,age,occupation,zipCode,title,genres,label
0,9,2294,4,978226678,1.0,590|480|2302|2268|529|349|524|16|1653|2278,M,25,17,61614,Antz (1998),Animation|Children's,1
1,15,3510,5,978361393,1.0,1892|648|1422|1597|3489|1196|260|1210|3461|2115,M,25,7,22903,Frequency (2000),Drama|Thriller,1
2,2165,3510,5,974982139,1.0,1095|2959|2416|3552|16|1266|3210|104|3147|3148,M,25,7,32836,Frequency (2000),Drama|Thriller,1
3,2319,3510,4,974476547,1.0,2987|1240|2243|1302|2716|1961|1096|3408|3481|3148,M,35,7,48104,Frequency (2000),Drama|Thriller,1
4,16,2701,2,978174795,1.0,2683|2699|2355|2975|2369|3175|2761|2581|2394|2724,F,35,0,20670,Wild Wild West (1999),Action|Sci-Fi|Western,0


In [8]:
print(train_df.shape, valid_df.shape)

(4832, 13) (1208, 13)


In [9]:
number_features = [
    Number('age', StandardScaler())
]

category_features = [
    Category('gender', CategoryEncoder(min_cnt=1)),
    Category('movieId', CategoryEncoder(min_cnt=1)),
    Category('occupation', CategoryEncoder(min_cnt=1)),
    Category('zipCode', CategoryEncoder(min_cnt=1))
]

sequence_features = [
    Sequence('genres', SequenceEncoder(sep='|', min_cnt=1)),
    Sequence('histHighRatedMovieIds', SequenceEncoder(sep='|', min_cnt=1))
]

In [10]:
features, train_loader, valid_loader = create_dataloader_fn(
    [], category_features, sequence_features, 64, train_df, 'label', valid_df, 4)

In [11]:
def evaluation(df, dataloader):
    preds = predict(model, dataloader)
    return roc_auc_score(df['label'], preds.ravel())

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

In [13]:
din_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1)]

gru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='GRU')]

aigru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AIGRU')]

models = [
    DNN(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DeepFM(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIN(features, din_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIEN(features, gru_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIEN(features, aigru_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3)
]

In [14]:
scores = []
for model in models:
    loss_func = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer, mode='min', factor=0.5, patience=5)
    fit(10, model, loss_func, optimizer,
        train_loader, valid_loader, scheduler, notebook=True)
    
    scores.append(evaluation(valid_df, valid_loader))

In [15]:
print(scores)

[0.5875926747712632, 0.5760858812123542, 0.5851957794290981, 0.5820781195677673, 0.6109539247747964]


In [16]:
for model in models:
    for child in model.children():
        print(child)
    print("-------------------------------")

Embedding(3, 16)
Embedding(1717, 16)
Embedding(22, 16)
Embedding(2956, 16)
Embedding(20, 16, padding_idx=0)
MaxPooling()
Embedding(2917, 16, padding_idx=0)
MaxPooling()
MLP(
  (_sequential): Sequential(
    (dense0): Linear(in_features=96, out_features=32, bias=True)
    (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation0): ReLU()
    (dropout0): Dropout(p=0.3)
    (dense1): Linear(in_features=32, out_features=16, bias=True)
    (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation1): ReLU()
    (dropout1): Dropout(p=0.3)
  )
)
Linear(in_features=16, out_features=1, bias=True)
-------------------------------
Embedding(3, 16)
Embedding(1717, 16)
Embedding(22, 16)
Embedding(2956, 16)
Embedding(20, 16, padding_idx=0)
MaxPooling()
Embedding(2917, 16, padding_idx=0)
MaxPooling()
FM()
MLP(
  (_sequential): Sequential(
    (dense0): Linear(in_features=96, out_features=32, bias=Tr